<h1 style="color:#1398A1">TRAINING<h1>


<h3 style="color:#CCC229">LIBRAIRIES IMPORT<h3>


In [20]:
import sys
from pathlib import Path

# Ajouter le dossier src au path
src_path = Path("..") / ".."
sys.path.append(str(src_path))
%load_ext autoreload
%autoreload 2

from datetime import datetime
import json
import uuid

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from env.snake_env import SnakeEnv
from utils.display import display_training_summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h3 style="color:#CCC229">TRAIN FUNCTION<h3>

In [21]:
import sys
def train_snake(
    timesteps: int = 100000,
    grid_size: int = 5,
    n_envs: int = 4,
    algorithm: str = "PPO",
    display_style: str = "glass"
):
    """
    Entraîne un agent Snake avec l'algorithme spécifié.

    Args:
        timesteps: Nombre total de timesteps d'entraînement
        grid_size: Taille de la grille (ex: 5 pour une grille 5x5)
        n_envs: Nombre d'environnements parallèles
        algorithm: Algorithme de RL à utiliser (actuellement seul 'PPO' est supporté)
        display_style: Style d'affichage ('gradient' ou 'minimal')

    Returns:
        tuple: (model, save_path, agent_uuid)
    """
    # Vérifie l'algorithme supporté
    if algorithm != "PPO":
        raise ValueError(f"Algorithme '{algorithm}' non supporté. Seul 'PPO' est disponible pour le moment.")

    # Crée un environnement vectorisé pour l'entraînement
    env = make_vec_env(lambda: SnakeEnv(grid_size=grid_size, render_mode=None), n_envs=n_envs)

    # Initialise le modèle selon l'algorithme
    if algorithm == "PPO":
        agent = PPO("MlpPolicy", env, verbose=1)

    # Entraîne le modèle
    agent.learn(total_timesteps=timesteps)

    # Prépare le dossier de sauvegarde avec sous-dossier par taille de grille
    repo_root = Path().resolve().parent.parent.parent
    save_dir = repo_root / "agent" / "agents" / f"{grid_size}x{grid_size}"

    # Crée le dossier s'il n'existe pas
    save_dir.mkdir(parents=True, exist_ok=True)

    # Génère un UUID unique pour cet agent
    agent_uuid = str(uuid.uuid4())

    # Nom du fichier avec date et UUID
    date_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    agent_name = f"agent_{agent_uuid}"

    # Sauvegarde l'agent
    save_path = save_dir / agent_name
    agent.save(str(save_path))

    # Chemin relatif depuis la racine du projet (portable pour Docker)
    relative_path = f"agent/agents/{grid_size}x{grid_size}/{agent_name}.zip"

    # Prépare les métadonnées du nouvel agent
    new_metadata = {
        "uuid": agent_uuid,
        "algorithm": algorithm,
        "grid_size": grid_size,
        "n_envs": n_envs,
        "total_timesteps": timesteps,
        "training_date": date_str,
        "agent_filename": f"{agent_name}.zip",
        "agent_path": relative_path
    }

    # Chemin du fichier JSON centralisé pour cette taille de grille
    metadata_file = save_dir / "agents_history.json"

    # Charge l'historique existant ou crée un nouveau
    if metadata_file.exists():
        with open(metadata_file, 'r') as f:
            history = json.load(f)
    else:
        # Crée un nouveau fichier d'historique
        history = {
            "grid_size": f"{grid_size}x{grid_size}",
            "agents": []
        }

    # Ajoute le nouvel agent à l'historique
    history["agents"].append(new_metadata)

    # Sauvegarde l'historique mis à jour
    with open(metadata_file, 'w') as f:
        json.dump(history, f, indent=4)

    # Affichage du résumé
    display_training_summary(
        agent_uuid=agent_uuid,
        algorithm=algorithm,
        grid_size=grid_size,
        timesteps=timesteps,
        n_envs=n_envs,
        relative_path=relative_path,
        style=display_style
    )

    return agent, str(save_path), agent_uuid

In [22]:
agent, path, agent_uuid = train_snake(timesteps=1000, grid_size=5, display_style="glass")

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9.69     |
|    ep_rew_mean     | -0.76    |
| time/              |          |
|    fps             | 9120     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 8192     |
---------------------------------
